# Constraints Analysis

In [1]:
# Add path to src/CARPy, in case notebook is running locally
import os, sys, warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))
warnings.simplefilter("ignore")  # <-- Suppress warnings

***
## Introduction

CARPy provides users with the ability to constrain designs, using a flavour of Energy-manoeuvrability theory.
To make the best use of this concept, the following objects are introduced:

1. `Constraint`: The base constraint class that provides constraint analysis methods.
2. `Constraints`: A means of considering multiple constraints together and operating on them.
3. `ThrustConstraint`: A type of design constraint for maximising vehicle range, based on Energy-manoeuvrability theory.
4. `PowerConstraint`: A type of design constraint for maximising vehicle endurance, based on Energy-manoeuvrability theory.

These objects are laid out in this way to facilitate future expansion of the constraint concept.

***
## 1) `Constraint` and 2) `Constraints` objects

We begin in a rather unorthodox manner - we introduce the generic types of `Constraint` and `Constraints`.
The purpose of these objects is to provide users with a convenient framework for defining a vehicle's mission in a regularised format.

They don't do anything by themselves when imported, but it's important to understand what they do for the objects that inherit their methods.

In [2]:
# These imports have to be made from a private file, as they are not
# designed to be directly imported by the user. You will in most cases,
# not have to do the following - this is all for illustrative purposes!
from carpy.conceptualdesign._constraints import Constraint, Constraints

#### `Constraint`

The main feature of a constraint is that it defines the `__call__` method - allowing sub-classes of `Constraint` to be called.
We'll get to how exactly calling works when we have an example constraint, but for now, accept that we can evaluate a constraint using `__call__`.

In [3]:
# Define a Constraint object
myconstraint = Constraint()

# Handle the error that we know we're about to raise
try:
    myconstraint()
except RuntimeError as e:
    # Show the reader the error we would've raised
    print(f"{type(e).__name__}: {e}")
finally:
    del myconstraint

RuntimeError: __call__ is only supported on children of <class 'carpy.conceptualdesign._constraints.Constraint'>


#### `Constraints`

Multiple `Constraint` objects can be packaged together in a `Constraints` (plural) object.

In [4]:
# Make two brand new constraints (for illustrative purposes only)
class MyChildConstraint(Constraint):
    pass
constraint1 = MyChildConstraint()
constraint2 = MyChildConstraint()

# Create two fictitious missions, based on different constraints
mission1 = Constraints(constraints=[constraint1])
mission2 = Constraints(constraints=[constraint2])

# See that missions concatenate their constraints with addition
print((mission1 + mission2).constraints)

(<__main__.MyChildConstraint object at 0x000001B035B19880>, <__main__.MyChildConstraint object at 0x000001B048D7C970>)


And that's all there is to it, for now.
To see how these work in practice, let's look at a real child of the `Constraint` class.

***
## 3) `ThrustConstraint(Constraint)` objects

The concept of an energy constraint is based on a model of aircraft performance based on Energy-manoeuvrability theory.
In short, the equations of motion are considered for a vehicle, and can then be resolved for any design variable.
CARPy's implementation makes the thrust-to-weight ratio, $\text{T/W}$, the objective.
For a detailed derivation, please see the [theory docs](https://github.com/yaseen157/carpy/tree/main/docs/source/theory).

Start with relevant imports

In [5]:
from carpy.conceptualdesign import ThrustConstraint
from carpy.environment import ISA1975

# Prescribe flight conditions at the design point
M = 0.5  # Mach number
z = 8e3  # 8,000 m altitude

# Create an atmosphere object, and use it to compute relevant parameters
myatmosphere = ISA1975()
speed = myatmosphere.c_sound(z, geometric=True) * M
dynamic_pressure = myatmosphere.rho(z, geometric=True) * speed ** 2

# Apply the computed dynamic pressure to our energy constraint
mymission = ThrustConstraint(q=dynamic_pressure)

The object instantiated from the `ThrustConstraint` class is actually an instance of `Constraints`, which in this case, contains a single object.
As discussed previously, this is a child of the `Constraint` class:

In [6]:
mymission.constraints

(<carpy.conceptualdesign._constraints.ThrustConstraint at 0x1b035af3b50>,)

In [7]:
constraint1, = mymission.constraints
print(
    f"constraint1 is of type '{type(constraint1).__name__}',"
    f" and is a child of the parent class '{type(constraint1).__base__.__name__}'."
)

constraint1 is of type 'ThrustConstraint', and is a child of the parent class 'Constraint'.


Each constraint contained with a `Constraints` object represents a single design point or goal.
But just as easily, we can revisit a design with a sequence of constraints - where the design conditions are broadcasted as required.

In [8]:
M = [0.5, 0.55]  # <-- Multiple Mach numbers
z = 8e3  # <-- Single altitude is broadcasted
zdot = [1.2, 0]  # <-- Multiple climb rates

# Compute the dynamic pressure, and apply it to our constraint object
speed = myatmosphere.c_sound(z, geometric=True) * M
dynamic_pressure = myatmosphere.rho(z, geometric=True) * speed ** 2
mymission = ThrustConstraint(q=dynamic_pressure, zdot=zdot)

# Multiple, individual constraints!
print(mymission.constraints)

(<carpy.conceptualdesign._constraints.ThrustConstraint object at 0x000001B04AC0D1C0>, <carpy.conceptualdesign._constraints.ThrustConstraint object at 0x000001B04AC40400>)


#### Governing Equation

The governing equation for a constraint is hidden in private variable. 

In [9]:
# The governing equation of a constraint (not designed for users to interface with)
print(ThrustConstraint._eqn)

Eq(T/W, (-C_D*S*q*(mu*sin(2*alpha)/2 - 1)/W - C_L*S*mu*q*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + zdot/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-C_D*S*mu*q*sin(alpha)**2/W - C_L*S*q*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + (1 - zdot**2/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))


Looks complicated!
This is a SymPy representation of the fundamental performance equation that governs this type of constraint.
If you're looking at this on a jupyter notebook, it should look nicer below:

In [10]:
# Jupyter notebooks support nicer formatting
ThrustConstraint._eqn

Eq(T/W, (-C_D*S*q*(mu*sin(2*alpha)/2 - 1)/W - C_L*S*mu*q*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + zdot/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-C_D*S*mu*q*sin(alpha)**2/W - C_L*S*q*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + (1 - zdot**2/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))

Taking steady level flight (and some other assumptions), the form might be more familiar to readers of aircraft conceptual design textbooks as:

In [11]:
import sympy as sp

# Steady level flight --> Lift == Weight
# Define some symbols so we can tell the solver W = L = q * S * C_L
W, S, q, C_L = sp.symbols("W,S,q,C_L")

# Apply our simplifications
ThrustConstraint._eqn.subs({
    "epsilon":0, "mu":0, "Vdot":0, "Vdot_n":0, "zdot":0, (q * S * C_L) : W})

Eq(T/W, C_D*S*q/(W*cos(alpha)))

#### In practice

We talked a little bit earier about the `__call__` method.
Officially, this is how a user is expected to make substitutions to the equation (instead of accessing a classes `._eqn` attribute.
As we applied a dynamic pressure and climb rate substitution earlier, we should see those values automatically propagate when we call our constraint without passing any arguments:

In [12]:
constraint1()

Eq(T/W, (-12477.8465402665*C_D*S*(mu*sin(2*alpha)/2 - 1)/W - 12477.8465402665*C_L*S*mu*cos(alpha)**2/W + Vdot/g + mu*cos(alpha)*cos(theta) + zdot/V)/((mu*cos(alpha)*tan(alpha + epsilon) + 1)*cos(alpha + epsilon)) + (-12477.8465402665*C_D*S*mu*sin(alpha)**2/W - 12477.8465402665*C_L*S*(mu*sin(2*alpha)/2 + 1)/W + Vdot_n/g + mu*cos(alpha)*cos(theta) + (1 - zdot**2/V**2)**0.5)/((mu*sin(alpha) + 1)*sin(alpha + epsilon)))

Looks pretty nasty!
We can drastically simplify the expression with more substitutions

In [13]:
constraint1(
    mu=0,  # Rolling resistance of zero, for a flying vehicle
    epsilon=0,  # Thrust setting angle of zero, thrust axis aligns with the vehicle longitudinal axis
    zdot=0,  # Rate of climb zero, steady altitude (overriding our defined climb rate)
    Vdot=0,  # Acceleration along flight path is zero (no linear acceleration)
    Vdot_n=0  # Acceleration normal to flight path is zero (no circular motion)
)

Eq(T/W, 12477.8465402665*C_D*S/(W*cos(alpha)) + (-12477.8465402665*C_L*S/W + 1)/sin(alpha))

That's better.
In this case the angle of attack $\alpha$, coefficients of lift $C_L$ and drag $C_D$, and wing-loading are all closely related and very inter-dependent parameters.
Let's make some more substitutions:

In [14]:
# Make the earlier substitutions stick for future
# calls, so we don't have to keep substituting them
constraint1.mu = 0
constraint1.epsilon = 0
constraint1.zdot = 0
constraint1.Vdot = 0
constraint1.Vdot_n = 0

# Take it one step further
constraint1.alpha=(1 * 3.141592/180)  # 1 degree angle of attack

# Show the equation in the notebook
constraint1()

Eq(T/W, 12479.7472654312*C_D*S/W - 714964.390772459*C_L*S/W + 57.2987004179959)

At the conceptual stage of design, the weight $W$ or reference wing area $S$ is not always known, but a ratio of the two (wingloading, $\text{W/S}$) might be.
We can substitute ratios into the SymPy expression of the constraint, by rearranging to make $W$ proportional to $S$:

In [15]:
import sympy as sp
T, W, S = sp.symbols("T,W,S")

# Define the wingloading proportionality
wingloading = {W: 2000 * S}

# Solve for T/W, then substitute in wingloading
eqn_t2w = sp.solve(constraint1(), T/W)[0]
eqn_t2w = eqn_t2w.subs(wingloading).simplify()

# Show the equation in the notebook
print(f"T/W = {eqn_t2w}")

T/W = 6.2398736327156*C_D - 357.48219538623*C_L + 57.2987004179959


In [16]:
# Parameters below come from guessing numbers that made a nice T/W.
# In a real-world scenario, a solver paired with an aerodynamic model is better.
lift_to_drag = 12
CL = 0.16

# Make the substitutions using a dictionary
t2w = eqn_t2w.subs({"C_L":CL, "C_D":CL / lift_to_drag})

# In this case, our desired value is on the right hand side of the governing equation equality
print(f"The computed Thrust-to-Weight ratio required at the design point is: T/W = {t2w:.3f}")

The computed Thrust-to-Weight ratio required at the design point is: T/W = 0.185


***
## 4) `PowerConstraint(Constraint)` objects

_Coming soon_